In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [2]:
all_users = pd.read_csv('CSVs/streamer_df_clean')
all_users

,username,AVG Viewers,Time Streamed (in hours),Hours Watched,Followers Gained,Total Followers,Total Views,All Time Peak Viewers
0,tommyinnit,195782,14.3,2806209,334307,5499713,49185807,650237
1,Quackity,158364,10.2,1620588,198959,3471874,23178122,381550
2,Sapnap,129001,9.2,1188960,183191,2237876,8740493,287152
3,auronplay,113520,109.4,12419097,341562,8741858,142994817,313347
4,xQcOW,109059,339.9,37072768,182238,5776761,361856811,222720
...,...,...,...,...,...,...,...,...
2445,SwordsofLegendsOnline,1237,18.7,23158,5165,11966,89491,4074
2446,easywithaces,1235,100.4,124074,1685,111557,8879377,18686
2447,SimpleFlips,1234,64.1,79125,1877,139942,5155937,8607
2448,Raptor,1233,148.0,182520,4138,171559,3312085,5270


In [3]:
all_users['followers_conversion'] = all_users['Total Followers'] / all_users['Total Views']

In [4]:
#df['followers_conversion'].plot()

In [5]:
def parse_games_2(user_name):
    user_name.lower()
    headers = {'User-Agent': 'Mozilla/5.0 (Platform; Security; OS-or-CPU; Localization; rv:1.4) Gecko/20030624 Netscape/7.1 (ax)'}
    response = requests.get(url = f"https://twitchtracker.com/{user_name}/games", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    df=pd.read_html(str(soup))[0]
    return df

In [6]:
df = parse_games_2('Shroud')

In [7]:
df.head()

,#,Game,Avg. viewers,Max. viewers,Followers,Total airtime,Last seen
0,#1,PLAYERUNKNOWN'S BATTLEGROUNDS,29080,152919,665 /hour,157330 27.1%,2021-04-02 06:00:00
1,#2,Counter-Strike: Global Offensive,13602,104421,309.4 /hour,40344 7%,2021-04-09 23:40:00
2,#3,Apex Legends,48327,202955,1185.3 /hour,36546 6.3%,2021-05-05 20:00:00
3,#4,World of Warcraft,18907,51344,262.4 /hour,35051 6%,2021-06-08 09:30:00
4,#5,VALORANT,54721,516289,1558.4 /hour,32646 5.6%,2021-06-01 21:50:00


In [8]:
def clean_gamer_df(username):
    df=parse_games_2(username)
    df["Total airtime"] = df["Total airtime"].astype(str).str[:-1]
    df[['Followers per hour', 'to_remove']]=df['Followers'].str.split(' /',expand=True)
    df = df.drop(columns=['Followers', 'to_remove'])
    df[['Total airtime','Percentage of total airtime']]=df['Total airtime'].str.split(' ',expand=True) 
    df['Percentage of total airtime'] = df['Percentage of total airtime'].apply(lambda x: float(x))
    df['Followers per hour'] = df['Followers per hour'].apply(lambda x: float(x))
    df['Last seen'] = pd.to_datetime(df['Last seen'])
    df=df.rename(columns={'#':'id'})
    df['id']=df['id'].astype(str).str[0]
    df=df.set_index('id')
    return df

In [9]:
sh = clean_gamer_df('tommyinnit')

In [10]:
sh.head()

,Game,Avg. viewers,Max. viewers,Total airtime,Last seen,Followers per hour,Percentage of total airtime
id,,,,,,,
#,Minecraft,30664,650237,64219,2021-05-31 20:30:00,1101.7,89.0
#,Just Chatting,20404,380838,4613,2021-05-14 21:20:00,1000.8,6.4
#,Among Us,67458,267938,1062,2021-02-10 22:00:00,2779.8,1.5
#,Fortnite,9,22,562,2019-02-02 19:10:00,0.9,0.8
#,PLAYERUNKNOWN'S BATTLEGROUNDS,29,445,428,2020-03-19 23:00:00,4.6,0.6


In [11]:
top_games_avg_viewers = sh.sort_values(by = ['Avg. viewers'], ascending=False)

In [12]:
#top 5 regarding AVERAGE VIEWERS
top_10 = top_games_avg_viewers.head(10)

In [38]:
fig = px.pie(top_10, values='Avg. viewers', names='Game', title='Average Viewers Top 10 Games', color_discrete_sequence=px.colors.sequential.RdBu)

In [39]:
fig.show()

In [15]:
ggrouped = sh.groupby('Game').sum().sort_values(by='Avg. viewers', ascending=False)

In [16]:
top_games_max_viewers_gr = ggrouped.sort_values(by = ['Max. viewers'], ascending=False)

In [17]:
top_10_grouped = top_games_max_viewers_gr.head(10)

In [18]:
top_games_pct_airtime = sh.sort_values(by =['Percentage of total airtime'], ascending=False)

In [19]:
top_5_airtime = top_games_pct_airtime.head()

In [20]:
fig1 = px.pie(top_5_airtime, values='Percentage of total airtime', names='Game', title='Airtime top 10 games', color_discrete_sequence=px.colors.sequential.RdBu)

In [21]:
fig1

In [22]:
top_games_followers_gained = sh.sort_values(by =['Followers per hour'], ascending=False)

In [23]:
top_10_followers_gained= top_games_followers_gained.head(10)

In [24]:
fig2=px.pie(top_10_followers_gained, values='Followers per hour', names='Game', title='Most followers gained',color_discrete_sequence=px.colors.sequential.RdBu)

In [25]:
fig2

In [26]:
#he is streaming mostly Battlegrounds
#he had the most viewers in Valorant
#he earned his followers in Marvels Avengers

In [27]:
sh['Date'] = sh['Last seen'].dt.date

In [28]:
sh.head()

,Game,Avg. viewers,Max. viewers,Total airtime,Last seen,Followers per hour,Percentage of total airtime,Date
id,,,,,,,,
#,Minecraft,30664,650237,64219,2021-05-31 20:30:00,1101.7,89.0,2021-05-31
#,Just Chatting,20404,380838,4613,2021-05-14 21:20:00,1000.8,6.4,2021-05-14
#,Among Us,67458,267938,1062,2021-02-10 22:00:00,2779.8,1.5,2021-02-10
#,Fortnite,9,22,562,2019-02-02 19:10:00,0.9,0.8,2019-02-02
#,PLAYERUNKNOWN'S BATTLEGROUNDS,29,445,428,2020-03-19 23:00:00,4.6,0.6,2020-03-19


In [29]:
px.histogram(sh,x='Date',y='Followers per hour')

In [30]:
px.bar(sh,x='Date',y='Avg. viewers')

In [31]:
px.histogram(sh,x='Date',y='Max. viewers')

In [32]:
#do this for 2021
px.scatter(sh, x='Date', y='Game', color='Game')

In [33]:
px.scatter(year2021, x='Date', y='Game', color='Game')

NameError: name 'year2021' is not defined

In [ ]:
sh

In [ ]:
sh['Date'].dtype

In [ ]:
sh['Date'] = pd.to_datetime(sh['Date'])

In [ ]:
sh['Date'].dtype

In [ ]:
sh.Game.value_counts()

In [ ]:
date_followers = sh.groupby('Date')[['Followers per hour']].sum()

In [ ]:
date_followers.reset_index

In [ ]:
px.line(date_followers)

In [ ]:
sh['time'] = sh['Last seen'].dt.time

In [ ]:
sh

In [ ]:
type(sh['time'])

In [ ]:
sh.sort_values(by=['time'])

In [ ]:
sh['time']=sh.time.astype(str).str[:2]

In [ ]:
sh

In [ ]:
time_freq = sh.groupby(['time'])[['Game']].count()

In [ ]:
px.line(time_freq)